### Import bibliotek

In [7]:
import requests
from bs4 import BeautifulSoup

### Wysłanie żądania dostępu do strony internetowej z opiniami o produkcie

In [ ]:
url = "https://www.ceneo.pl/84514582#tab=reviews"
response = requests.get(url)
response.status_code

### Jeżeli operacja zakończy się powodzeniem

In [24]:
if response.status_code == 200:
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")

### Dla każdej opinii wyodrębnienie z kodu HTML poszczególnych składowych i przypisanie ich do elementów złożonej struktury danych

In [ ]:
for opinion in opinions:
    try:
        single_opinion = {
            "Author": opinion.select_one("span.user-post__author-name").text.strip(),
            "Recommendation": opinion.select_one("span.user-post__author-recomendation > em").text.strip(),
            "Stars": opinion.select_one("span.user-post__score-count").text.strip(),
            "Content": opinion.select_one("div.user-post__text").text.strip(),
            "Pros": [p.text.strip() for p in opinion.select_one("div.review-feature__item--positive")],
            "Cons": [c.text.strip() for c in opinion.select_one("div.review-feature__item--negative")],
            "Useful": opinion.select_one("button.vote-yes > span").text.strip(),
            "Useless": opinion.select_one("button.vote-no > span").text.strip(),
            "Post_date": opinion.select_one("span.user-post__published > time:nth-child(1)[datetime]").text.strip(),
            "Purchase_date": opinion.select_one("span.user-post__published > time:nth-child(2)[datetime]").text.strip(),
    }
        print(single_opinion)
    except (AttributeError, TypeError):
        pass

In [ ]:
print(type(single_opinion["author"]))